#  Orientation data - angle analysis 
Analysis of the angles in 2D, then possibly 3D.
Goal: find angular velocity to compare with event log. 

In [3]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline
%matplotlib notebook

path = '../datasets/'
fname = 'VRAcuityExp_2017-07-13_14-39-17_VR-4A_NIC_clean.h5'
rat_orientation = pd.read_hdf(path+fname, 'Orientation')

In [2]:
#dropping the y to go to 2D unitvector and then normalize - probably bad approach
i=20000

def create_vec(X, Y, Z, i):
    A = np.array([X[i],Y[i],Z[i]])
    return A

def create_flat_vec(X, Y, Z, i):
    A = np.array([X[i],Y[i],Z[i]])
    B = np.array([X[i],0   ,Z[i]])

    Bm = np.linalg.norm(A)
    B1 = B / Bm
    #print(B, B1)
    return B1

def angle_vec(V1, V2):
    CosTheta = np.dot(V1,V2) / (np.linalg.norm(V1) * np.linalg.norm(V2))
    Theta = np.arccos(CosTheta)
    return CosTheta, Theta

X, Y, Z, Time = rat_orientation.X,rat_orientation.Y, rat_orientation.Z, rat_orientation.Time
dt = Time[i+1] - Time[i]

# XYZ vector angular speed
V1 = create_vec(X, Y, Z, i)
V2 = create_vec(X, Y, Z, i+1)

CosTheta, Theta = angle_vec(V1, V2)
U = Theta/dt
print('XYZ',CosTheta, Theta)

# y -> to zero
XZ1 = create_flat_vec(X, Y, Z, i)
XZ2 = create_flat_vec(X, Y, Z, i+1)

CosTheta, Theta = angle_vec(XZ1, XZ2)
U = Theta/dt
print('XZ', CosTheta, Theta)

#y vector angular speed
Y1 = np.array([0, Y[i]  , 1])
Y2 = np.array([0, Y[i+1], 1])

CosTheta, ThetaY = angle_vec(Y1, Y2)
UY = ThetaY/dt
print('Y', CosTheta, ThetaY)


NameError: name 'rat_orientation' is not defined

In [61]:
for i in range(8000,10000):
    dt = Time[i+1] - Time[i]
    # XYZ vector angular speed
    V1 = create_vec(X, Y, Z, i)
    V2 = create_vec(X, Y, Z, i+1)

    CosTheta, Theta = angle_vec(V1, V2)
    U = Theta/dt
    #print('XYZ',Theta)
    #print('XYZ', CosTheta)

    # y -> to zero
    XZ1 = create_flat_vec(X, Y, Z, i)
    XZ2 = create_flat_vec(X, Y, Z, i+1)

    CosThetaXZ, ThetaXZ = angle_vec(XZ1, XZ2)
    U = ThetaXZ/dt

    #y vector angular speed
    Y1 = np.array([0, Y[i]  ,Z[i] ])
    Y2 = np.array([0, Y[i+1],Z[i+1]])

    CosThetaY, ThetaY = angle_vec(Y1, Y2)
    UY = ThetaY/dt
    #print('XZ', Theta, U, 'Y', ThetaY, UY)
    print('XZ', CosThetaXZ, 'Y', CosThetaY, 'Theta:XZ', ThetaXZ, 'Y', ThetaY)
    
    

XZ 0.999985555226 Y 0.99999725077 Theta:XZ 0.00537490620965 Y 0.00234487997891
XZ 0.999992692251 Y 0.999999950794 Theta:XZ 0.00382302455094 Y 0.000313708324481
XZ 0.999972100781 Y 0.999998796987 Theta:XZ 0.00746985253929 Y 0.00155113709349
XZ 0.999977267409 Y 0.999999458573 Theta:XZ 0.00674280011491 Y 0.00104060241072
XZ 0.999936757461 Y 0.999991417103 Theta:XZ 0.0112466177731 Y 0.00414316532256
XZ 0.992860713125 Y 0.993353831795 Theta:XZ 0.119564191859 Y 0.115356345349
XZ 0.999793648634 Y 0.998900477356 Theta:XZ 0.0203154356869 Y 0.0468982770414
XZ 0.986907806616 Y 0.986358745689 Theta:XZ 0.16199296979 Y 0.165362520523
XZ 0.999995415251 Y 0.999997411727 Theta:XZ 0.00302811902267 Y 0.00227520279574
XZ 0.999999826852 Y 0.999996344104 Theta:XZ 0.000588469910576 Y 0.00270403344265
XZ 0.991410338522 Y 0.993375358237 Theta:XZ 0.131163918593 Y 0.11516917144
XZ 0.998992199108 Y 0.998971966635 Theta:XZ 0.0448992252732 Y 0.0453477574993
XZ 0.995734658525 Y 0.996186868025 Theta:XZ 0.092394557396

XZ 0.999974913876 Y 0.999993167724 Theta:XZ 0.00708325197111 Y 0.00369656147445
XZ 0.999993428633 Y 0.999999672886 Theta:XZ 0.00362529278923 Y 0.000808843572144
XZ 0.999876930856 Y 0.999775778415 Theta:XZ 0.0156889558751 Y 0.0211768724916
XZ 0.99999916535 Y 0.999999754167 Theta:XZ 0.00129201383728 Y 0.000701189283345
XZ 0.999977018322 Y 0.999983678556 Theta:XZ 0.00677964100099 Y 0.0057134031859
XZ 0.999997373828 Y 0.99998251808 Theta:XZ 0.00229179997711 Y 0.0059130315304
XZ 0.99998898416 Y 0.999997348356 Theta:XZ 0.00469379604753 Y 0.00230288723947
XZ 0.999983685817 Y 0.99999951189 Theta:XZ 0.00571213217978 Y 0.000988038268467
XZ 0.999984448149 Y 0.999945912494 Theta:XZ 0.00557707646138 Y 0.0104007685467
XZ 0.999845833207 Y 0.999973877796 Theta:XZ 0.0175596557022 Y 0.00722804511214
XZ 0.999991734595 Y 0.999998836638 Theta:XZ 0.00406581271792 Y 0.00152536023118
XZ 0.999983608107 Y 0.999989711248 Theta:XZ 0.0057257204735 Y 0.00453624725967
XZ 0.999827294428 Y 0.999967539257 Theta:XZ 0.01

KeyError: 8350

In [16]:
# rotation update
# thetaP = theta + dt /2 * omega * theta
# omega = thetaP - theta / (dt/2 * theta)

i = 40000

def omega(theta, thetaP, dt):
    return thetaP - theta / (dt/2 * theta)

dt = Time[i+1] - Time[i]
V1 = create_vec(X, Y, Z, i)
V2 = create_vec(X, Y, Z, i+1)


# go to quaterions first, then check

print(dt)
print(omega(V1, V2, dt))

print(V2-V1)

0.004167
[-480.00794735 -480.66994191 -480.66595263]
[ 0.00279911  0.00015107 -0.00034176]


In [ ]:
# get theta, go to quaternion, then extract the omega
i = 40000


def quat(theta, x, y, z):
    Q = np.array([    np.cos(theta/2),
                  x * np.sin(theta/2),
                  y * np.sin(theta/2),
                  z * np.sin(theta/2)])
    return Q

dt = Time[i+1] - Time[i]
V1 = create_vec(X, Y, Z, i)
V2 = create_vec(X, Y, Z, i+1)

CosTheta = np.dot(V1,V2) / (np.linalg.norm(V1) * np.linalg.norm(V1))
Theta = np.arccos(CosTheta)
print(dt, Theta)


Q1 = quat(Theta, X[i], Y[i], Z[i])
Q2 = quat(Theta, X[i+1], Y[i+1], Z[i+1])

print(omega(Q1, Q2, 0.1))


## Second approach

In [10]:
def angle_vec(V1, V2):
    CosTheta = np.dot(V1,V2) / (np.linalg.norm(V1) * np.linalg.norm(V2))
    Theta = np.arccos(CosTheta)
    return CosTheta, Theta

X, Y, Z, Time = rat_orientation.X,rat_orientation.Y, rat_orientation.Z, rat_orientation.Time
dt = Time[i+1] - Time[i]
i = 100

# get two vectors aligned with Y and Z axes
Vo = np.array([X[i], Y[i], Z[i]])  
Vz = np.array([X[i], Y[i], 0   ])  
Vy = np.array([X[i], 0   , Z[i]])
Vx = np.array([0   , Y[i], Z[i]])

# find the angles between them and orientation vector 
cosAlpha, alpha = angle_vec(Vo, Vz)
cosBeta , beta  = angle_vec(Vo, Vy)
cosGamma, gamma = angle_vec(Vo, Vx)

print(alpha, beta, gamma)

# cos -> get lenghts of decomposition
LenghtVz = cosAlpha
LenghtVy = cosBeta
LenghtVx = cosGamma

# find the tip point with flat Y
Vxy = np.array([LenghtVx, LenghtVz])
print(Vxy)


## ADDITIONAL FILTERING BY THE TOO BIG Y?

0.444837719905 0.142023711661 1.10067573692
-0.891513668213 0.141546739192
[ 0.4529938   0.90268051]


In [16]:
# function def
def decompose_vec(X, Y, Z, i):
    # get two vectors aligned with Y and Z axes
    Vo = np.array([X[i], Y[i], Z[i]])  
    Vz = np.array([X[i], Y[i], 0   ])  
    Vy = np.array([X[i], 0   , Z[i]])
    Vx = np.array([0   , Y[i], Z[i]])

    # find the angles between them and orientation vector 
    LenghtVz, alpha = angle_vec(Vo, Vz)
    LenghtVy, beta  = angle_vec(Vo, Vy)
    LenghtVx, gamma = angle_vec(Vo, Vx)
 
    # find the tip point with flat Y
    Vxy = np.array([LenghtVx, 0, LenghtVz])
    return Vxy, LenghtVy

# analyzie two separete angular velocities based on that 
i = 200

dt = Time[i+1] - Time[i]
Vxy1, Y1 = decompose_vec(X, Y, Z, i  )
Vxy2, Y2 = decompose_vec(X, Y, Z, i+1)

cosTheta, theta = angle_vec(Vxy1, Vxy2)
U = theta/dt
print(U)

UY1 = (Y1-Y2)/dt
print(UY1)



0.623716629169
0.215101265011 0.0


In [18]:
#function in the loop

for i in range(2000,4000):
    Vxy1, Y1 = decompose_vec(X, Y, Z, i  )
    Vxy2, Y2 = decompose_vec(X, Y, Z, i+1)

    cosTheta, theta = angle_vec(Vxy1, Vxy2)
    U   = theta / dt    
    UY1 = (Y1-Y2) / dt
    print(U, UY1)
    

1.33260296029 0.480353052281
1.5237065653 0.445576531409
1.52188958428 0.378276644655
1.66302082665 0.439302474934
1.870146619 0.435210188003
1.69032558548 0.415485157744
1.71839120529 0.539115758567
1.92323005776 0.588717792858
1.99176468004 0.665297005026
2.05715276173 0.691999365343
1.78878694199 0.293828760222
1.70562218099 0.685618630639
1.81802493439 0.607694782226
1.63206425018 0.554949533619
1.9834465704 0.766256514378
1.48407729107 0.53954862579
1.67442841527 0.695131466763
1.4730651044 0.6345625267
0.802372954257 0.561857651231
0.931559236964 0.578651501865
1.11535416798 0.572271921216
1.20141158421 0.556155836674
0.938002538843 0.50369758731
0.972105372639 0.419445386472
0.787764371635 0.547583965529
0.824478169422 0.331211085837
0.76144432328 0.539666028787
0.59161966631 0.338187299548
0.590827424233 0.475557990755
0.869495583165 0.680413786256
0.632260684954 0.633138109882
0.664032326582 0.70673217858
0.590202846447 0.387550487947
0.73383917056 0.512443995032
0.70764434386

0.214355043387 -0.519550944912
5.09262625829 8.14008318439
0.273812678536 -0.507391018662
0.196335393098 -0.18273578053
0.521549638984 -0.707683099651
0.929258815791 -1.63212460064
0.873421138158 -1.29653133582
0.522648395959 -1.06471231502
2.0905639642 3.59324272645
0.520409602856 0.286909568737
4.34035099841 5.84075461761
1.31171769384 1.26743722603
0.848970689693 0.780671545959
0.457137259692 0.242898609676
0.222110026782 0.091531874013
0.222254268226 0.053473171398
0.0527350093447 -0.242324599729
0.219515431198 0.140887431931
0.427967352454 0.412433238712
0.451094196392 0.170514780118
0.435715507917 0.0176682261513
0.697405272759 0.242676872585
0.765529838422 0.162902094684
0.863784997362 0.176503169192
0.891332681039 0.100526805116
0.799625061527 -0.347566175884
0.686607493789 -0.769829890457
1.15653230704 -0.326578349281
0.85140070148 -0.766662220707
0.9924653308 -0.431873258884
0.317852114128 -1.73467896728
0.0714129305841 -1.84717533948
0.274737876737 -0.70465401796
0.569293966

KeyError: 2967